<a href="https://colab.research.google.com/github/martinpius/Computer-Vission/blob/main/Simple_Resnet_using_model_subclassing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("/content/drive", force_remount = True)
try:
  COLAB = True
  import tensorflow as tf
  print(f'You are on CoLaB with tensorflow version: {tf.__version__}')
except Exception as e:
  print(f"{type(e)}: {e}\nplease load your drive...")
def time_fmt(t:float=124.928)->float:
  h = int(t / (60 * 60))
  m = int(t % (60 * 60) / 60)
  s = int(t % 60)
  return f"{h}: {m:>03}: {s:>05.2f}"
print(f"display formated time....\n>>> time elapse: {time_fmt()}")

Mounted at /content/drive
You are on CoLaB with tensorflow version: 2.4.1
display formated time....
>>> time elapse: 0: 002: 04.00


In [2]:
#In this notebook we are going to construct the residual network using keras layer's subclassing and fit on cifar10 data:

In [3]:
import tensorflow as tf
import numpy as np
import time, os
import tensorflow.keras.backend as K

In [4]:
#We start by constructing the convolutional block:


In [5]:
class ConvBlock(tf.keras.layers.Layer):
  def __init__(self, num_filters, kernel = 3, *args, **kwargs):
    super(ConvBlock, self).__init__(*args, **kwargs)
    self.conv = tf.keras.layers.Conv2D(filters = num_filters,
                                       kernel_size = (kernel, kernel), 
                                       padding = 'same', 
                                       activation = 'relu')
    self.bn = tf.keras.layers.BatchNormalization()
  def call(self, inputs, training = False):
    x = self.conv(inputs)
    x = self.bn(x)
    return x 

In [6]:
#The residual block:

In [7]:
class ResNetBlock(tf.keras.layers.Layer):
  def __init__(self, num_filters, *args, **kwargs):
    super(ResNetBlock, self).__init__(*args, **kwargs)
    self.cnn1 = ConvBlock(num_filters = num_filters[0])
    self.cnn2 = ConvBlock(num_filters = num_filters[1])
    self.cnn3 = ConvBlock(num_filters = num_filters[2])
    self.max_pool = tf.keras.layers.MaxPooling2D()
    self.id_map = tf.keras.layers.Conv2D(filters = num_filters[1], kernel_size = 3, padding = 'same')
  
  def call(self, inputs_tensor, training = False):
    x = self.cnn1(inputs_tensor, training = training)
    x = self.cnn2(x, training = training)
    x = self.cnn3(x + self.id_map(inputs_tensor), training = training)
    return self.max_pool(x)

In [8]:
#Now we can define the simple residual model block

In [9]:
class ResModel(tf.keras.models.Model):
  def __init__(self, class_labels = 10, *args, **kwargs):
    super(ResModel, self).__init__(*args, **kwargs)
    self.block1 = ResNetBlock([32,64,128])
    self.block2 = ResNetBlock([64,128,256])
    self.block3 = ResNetBlock([128,256,512])
    self.block4 = ResNetBlock([256,256,128])
    self.block5 = ResNetBlock([64,128,256])
    self.flatten = tf.keras.layers.Flatten()
    self.dense1 = tf.keras.layers.Dense(units = 1024, activation = 'relu')
    self.dropout = tf.keras.layers.Dropout(rate = 0.5)
    self.dense2 = tf.keras.layers.Dense(units = 512, activation = 'relu')
    self.outputs = tf.keras.layers.Dense(units = 10, activation = 'softmax')

  def call(self, inputs_tensor, training = False):
    x = self.block1(inputs_tensor, training = training)
    x = self.block2(x, training = training)
    x = self.block3(x, training = training)
    x = self.block4(x, training = training)
    x = self.block5(x, training = training)
    x = self.flatten(x)
    x = self.dense1(x, training = training)
    x = self.dropout(x)
    x = self.dense2(x, training = training)
    x = self.outputs(x)
    return x

In [10]:
#Instantiate the model class:
model = ResModel(10)


In [11]:
#Get the data from keras:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
img_rows, img_cols = 32, 32
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 3, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 3, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 3)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 3)
    input_shape = (img_rows, img_cols, 3)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255


170500096/170498071 [==============================] - 3s 0us/step


In [12]:
print(f"x_train_shape: {x_train.shape}, x_test_shape: {x_test.shape}\ny_train_shape: {y_train.shape}, y_test_shape: {y_test.shape}")

x_train_shape: (50000, 32, 32, 3), x_test_shape: (10000, 32, 32, 3)
y_train_shape: (50000, 1), y_test_shape: (10000, 1)


In [13]:
y_train, y_test = tf.keras.utils.to_categorical(y_train, num_classes = 10), tf.keras.utils.to_categorical(y_test, num_classes = 10)

In [14]:
#Convert the data into tensorflow data type:

In [15]:
BUFFER_SIZE = len(x_train)
BATCH_SIZE = 64
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.shuffle(BUFFER_SIZE).batch(batch_size = BATCH_SIZE, drop_remainder = True)
validation_data = tf.data.Dataset.from_tensor_slices((x_test, y_test))
validation_data = validation_data.shuffle(BUFFER_SIZE).batch(batch_size = BATCH_SIZE, drop_remainder = True)
x_train_sample_batch, y_train_sample_batch = next(iter(train_data))
print(f"x_train_batch_shape: {x_train_sample_batch.shape}, y_train_sample_batch: {y_train_sample_batch.shape}")

x_train_batch_shape: (64, 32, 32, 3), y_train_sample_batch: (64, 10)


In [16]:
#the training loop from scratch:

In [17]:
EPOCHS = 25
tic = time.time()
optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001)
loss_obj = tf.keras.losses.CategoricalCrossentropy()
train_metric = tf.keras.metrics.CategoricalAccuracy()
val_metric = tf.keras.metrics.CategoricalAccuracy()
for epoch in range(EPOCHS):
  print(f"....training begin for epoch: {epoch+ 1}.....\n>>>please wait.....")
  for (step, (x_train_batch, y_train_batch)) in enumerate(train_data):
    with tf.GradientTape() as tape:
      preds = model(x_train_batch, training = True)
      train_loss = loss_obj(y_train_batch, preds)
    grads = tape.gradient(train_loss, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    train_metric.update_state(y_train_batch, preds)
    train_acc = train_metric.result()
    train_metric.reset_states()
    if step % 200 == 0:
      print(f"epoch: {epoch + 1}, train accuracy: {float(train_acc):.4f}")
      print(f"at batch {step}, train loss: {float(train_loss):.4f}")
  for (step, (x_val_batch, y_val_batch)) in enumerate(validation_data):
    preds = model(x_val_batch, training = False)
    val_loss = loss_obj(y_val_batch, preds)
    val_metric.update_state(y_val_batch, preds)
    val_acc = val_metric.result()
    val_metric.reset_states()
    if step % 200 == 0:
      print(f"epoch: {epoch + 1}, validation accuracy: {float(val_acc):.4f}")
      print(f"at batch {step}: validation_loss: {float(val_loss):.4f}")
toc = time.time()
print(f"\n>>>train and evaluation complited in {time_fmt(toc - tic)}")

....training begin for epoch: 1.....
>>>please wait.....
epoch: 1, train accuracy: 0.1406
at batch 0, train loss: 2.5929
epoch: 1, train accuracy: 0.3906
at batch 200, train loss: 1.4943
epoch: 1, train accuracy: 0.6406
at batch 400, train loss: 1.0989
epoch: 1, train accuracy: 0.6406
at batch 600, train loss: 0.9182
epoch: 1, validation accuracy: 0.6406
at batch 0: validation_loss: 1.1621
....training begin for epoch: 2.....
>>>please wait.....
epoch: 2, train accuracy: 0.6562
at batch 0, train loss: 0.8841
epoch: 2, train accuracy: 0.7344
at batch 200, train loss: 0.8368
epoch: 2, train accuracy: 0.7969
at batch 400, train loss: 0.6972
epoch: 2, train accuracy: 0.6875
at batch 600, train loss: 0.8534
epoch: 2, validation accuracy: 0.7031
at batch 0: validation_loss: 0.9734
....training begin for epoch: 3.....
>>>please wait.....
epoch: 3, train accuracy: 0.7812
at batch 0, train loss: 0.6865
epoch: 3, train accuracy: 0.7812
at batch 200, train loss: 0.5891
epoch: 3, train accuracy: 0